In [0]:
entity_str=[]
for substr in dbutils.widgets.get("entity_name").split("_"):
    if substr.startswith('inc'):
        entity_str.append(substr)
    else:
        entity_str.append(substr.capitalize())
    

entity_name = '_'.join(entity_str)

In [0]:
if entity_name.endswith('incremental'):
    load_type = 'incremental'
else:
    load_type = 'full'

In [0]:
%run ./CommonUtils

In [0]:
if load_type == 'full':
    src_df = spark.read.table(f"hive_metastore.{get_catalog_name(entity_name,'bronze')}.{entity_name.lower().split('_inc')[0]}")
    crt_execute(entity_name,'bronze',truncate_flag=False,src_df=src_df)
else:
    pk = pk_map_bronze(entity_name.split('_inc')[0].lower())
    SCD2(entity_name,pk=pk,ingestion_layer="silver").execute_scd_2()

In [0]:
print("Silver Ingestion Completed")